# Ensembling_on 6 datasets

# import data

In [1]:
import pandas as pd
data=pd.read_csv('ARIMA_predict.csv')
data1=pd.read_csv('Windowing_predict.csv')
data2=pd.read_csv('GBoostRegression_predict.csv')
data3=pd.read_csv('XGBoostRegression_predict.csv')
data4=pd.read_csv('CombineModel_predict1.csv')
data5=pd.read_csv('CombineModel_predict2.csv')

# merge data

In [2]:
data = data.rename(columns={'pre_units': 'pre_units1'})
data1 = data1.rename(columns={'pre_units': 'pre_units2'})
data2 = data2.rename(columns={'pre_units': 'pre_units3'})
data3 = data3.rename(columns={'pre_units': 'pre_units4'})
data4 = data4.rename(columns={'pre_units': 'pre_units5'})
data5 = data5.rename(columns={'pre_units': 'pre_units6'})


In [3]:
data=data.drop(['Unnamed: 0','units'],axis=1)
data1=data1.drop(['Unnamed: 0'],axis=1)
data2=data2.drop(['Unnamed: 0','date','units'],axis=1)
data3=data3.drop(['Unnamed: 0','date','units'],axis=1)
data4=data4.drop(['Unnamed: 0','units'],axis=1)
data5=data5.drop(['Unnamed: 0','units','date'],axis=1)

In [4]:
train_data=pd.merge(data1,data,how='left',left_on=['key','day'], right_on = ['key','day'])
train_data1=pd.merge(train_data,data2,how='left',left_on=['key','day'],right_on = ['key','day'])
train_data2=pd.merge(train_data1,data3,how='left',left_on=['key','day'], right_on = ['key','day'])
train_data3=pd.merge(train_data2,data4,how='left',left_on=['key','day'], right_on = ['key','day'])
train_data4=pd.merge(train_data3,data5,how='left',left_on=['key','day'], right_on = ['key','day'])
train_data=train_data4

In [5]:
train_data.head()

,key,day,units,pre_units2,pre_units1,pre_units3,pre_units4,pre_units5,pre_units6
0,19671 39 1/3,1,0.0,0.032266,0.082668,0.02940,0.01844,0.082668,0.082668
1,19671 39 1/3,2,0.0,0.034279,0.079272,0.03128,0.02018,0.079272,0.079272
2,19671 39 1/3,3,0.0,0.037971,0.076319,0.02303,0.01262,0.076319,0.076319
3,19671 39 1/3,4,0.0,0.036492,0.073751,0.02303,0.03937,0.073751,0.073751
4,19671 39 1/3,5,0.0,0.040193,0.071518,0.02303,0.01634,0.071518,0.071518


# ensembling model

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import numpy as np
import re
import sklearn
import xgboost as xgb
from xgboost import XGBRegressor

model = XGBRegressor(colsample_bytree=0.7,learning_rate=0.03,n_estimators=1000,max_depth=6,min_child_weight=7,objective='reg:linear',subsample=0.7)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
target_prediction = cross_val_predict(model, train_data.drop(['units','key','day'],axis=1), train_data[['units']], cv=cv)

/Users/xiaqian/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/xiaqian/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [8]:
df_prediction1=pd.DataFrame(target_prediction, columns=['pre_units']).round(decimals=5)

In [9]:
df_prediction_need=train_data[['key','day','units']]
df_prediction_need=df_prediction_need.reset_index(drop=True)
pre_total=df_prediction_need.merge(df_prediction1, left_index=True, right_index=True)

In [10]:
pre_total.shape

(397544, 4)

# measurement

In [11]:
import datetime

def predict_date(pre_total):
    date=[]
    i=0
    n=len(pre_total)
    while i<n:
        day_int=pre_total.iloc[i]['day']
        date_time=datetime.datetime(2018, 1, day_int, 0,0).date()
        date_string=str(date_time)
        date.append(date_string)
        i+=1
    pre_total['date']=date

    pre_total_pvt = pre_total.pivot_table('pre_units', ['key'], 'date')
    pre_total_pvt['key'] = pre_total_pvt.index
    pre_total_pvt=pre_total_pvt.reset_index(drop=True)

    return pre_total_pvt

In [12]:
pre_total_pvt= predict_date(pre_total)

In [13]:
from datetime import date, timedelta
import math

def measurement(prediction, test_data):

    d1 = date(2018, 1, 1)  
    d2 = date(2018, 1, 31)  
    delta = d2 - d1         
    pre_day=[]
    for i in range(delta.days + 1):
        add=str(d1 + timedelta(days=i))
        pre_day.append(add)

    sold_out_day=[]
    key=[]
    #lets innet join with testdataset
    window_inner=test_data.merge(prediction,left_on='key', right_on='key', how='inner')
    window_inner['sol_out_pred']=0

    all_=window_inner.key.unique()
    n=len(all_) 
    i=0

    while(i<n): #dataset
        subset_w=prediction.loc[prediction['key']==all_[i]]
        stock=test_data.loc[test_data['key']==all_[i]]['stock'].values[0]
        n1=len(pre_day)
        k=0
        while(k<n1): #day calculation
            day=pre_day[k]
            that_day_sale=subset_w[day].values[0]
            stock=stock-that_day_sale
            if((stock==0) | (stock<0) ):
                window_inner.iloc[i,37]=day
                break
            if(k==(n1-1)): 
                window_inner.iloc[i,37]='2018-01-22'
            k=k+1
        i=i+1

    test_data['key']=test_data["pid"].astype(str) +" "+ test_data["size"].astype(str)
    window_inner["date_sold_out_date"] = pd.to_datetime(window_inner["sold_out_date"],format="%Y/%m/%d")
    window_inner["date_sold_out_pred"] = pd.to_datetime(window_inner["sol_out_pred"],format="%Y/%m/%d")

    days_differ=window_inner["date_sold_out_date"]-window_inner["date_sold_out_pred"]
    days=pd.DataFrame(days_differ,columns=['days_differ'])
    days['correct_differ']=abs(days['days_differ'].astype(datetime.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days))

    error=math.sqrt(sum(days['correct_differ']))

    return error



In [14]:
def error(test_table,total):
    
    test_data_0=pd.read_csv(test_table)
    test_data_0['pid']=test_data_0['pid'].astype(int)
    test_data_0['key']=test_data_0["pid"].astype(str) +" "+ test_data_0["size"].astype(str)

    error_total=measurement(total, test_data_0)

    print('\ntotoal',error_total)
    
    return error_total
    

In [15]:
error_0=error('test_0.csv',pre_total_pvt)


totoal 252.01984048879962


In [16]:
error_1=error('test_1.csv',pre_total_pvt)


totoal 253.4896447589132


In [17]:
error_2=error('test_2.csv',pre_total_pvt)


totoal 253.7715508089904


In [18]:
error_3=error('test_3.csv',pre_total_pvt)


totoal 253.11459855172322


In [19]:
error_4=error('test_4.csv',pre_total_pvt)


totoal 253.0375466210499


In [20]:
import numpy

total=[error_0,error_1,error_2,error_3,error_4]

print(numpy.mean(total))

253.086636246
